# **페이지 처리 또는 부분범위처리 활용**

- 데이터를 전송하는 맨 마지막 단게에 함수 호출이 일어나도록 함

# **Decode 함수 또는 Case 문으로 변환하는 경우**

- 다량의 함수 호출을 피할 수 없을때
- 함수로직을 풀어서 decode 문이나 case문으로 전환하거나 조인문으로 구현할 수 있는지 확인.

# **뷰 머지(View Merge)방지를 통한 함수 호출 최소화**

- 함수를 풀어 조인문으로 변경하기 곤란한 경우 함수 호출 횟수를 줄이려 노력

# **스칼라 서브쿼리의 캐싱효과를 이용한 함수 호출 최소화**

- 함수로직이 복잡하거나 함수를 사용해야 하는 경우
  - 입력되는 값의 종류에 따라 소량인 경우 함수를 그대로 둔채 스칼라 서브쿼리 캐싱 효과를 이용.
- 스칼라 서브쿼리를 사용하면 오라클은 그 수행횟수를 최소화 하려고 입력값과 출력값을 내부 캐시에 저장.
  - 서브쿼리가 수행될 때 마다 입력값을 캐시에서 찾아보고 있으면 저장된 출력값을 return , 없으면 쿼리수행 후 입력값과 출력값을 캐시에 저장.
- 함수를 Dual 테이블을 이용해 스칼라 서브쿼리로 한번 감싸면 함수 호출횟수를 줄이는데 사용할 수 있음.
- 함수입력값의 종류가 소수여서 해시충돌 가능성이 적은 경우에만 사용하며 그렇지 않을 경우 CPU 낭비를 초래.
- 해시충돌이 발생하는 경우 Hidden Parameter 값인 _query_execution_cache_max_size 값을 증가시켜 개선이 가능.(10g)
- 스칼라 서브쿼리의 입/출력 값은 UGA에 저장되므로 Fetch Call과 상관없이 캐싱되는 순간부터 끝까지 유지.
  - UGA(User Global Area) : 세션과 관련된 정보를 저장하기 위해 사용되는 곳으로 Fixed UGA와 Variable UGA로 구분.
    - Fixed UGA :: 세션에 대한 기본 정보 및 Variable UGA에 대한 포인터를 가짐.
    - Variable UGA : Heap으로 구성.

# **Deterministic**

- 10gR2에서 함수를 선언할 때 Deterministic 키워드를 넣어 주면 스칼라 서브쿼리를 덧입히지 않아도 캐싱효과가 생김.
- 함수의 입력/출력값은 CGA에 캐싱되며 이 값은 데이터베이스 Call에서만 유효하여 Fetch Call이 완료되면 모두 해제.
- CGA(Call Global Area) : Call과 관련된 정보를 저장하기 위한 임시적인 공간이며, call이 시작될때 할당되고 call이 완료되면 해제.
  - 함수안에 쿼리문장을 포함하고 있다면 그 함수는 일관성이 보장되지 않음.
  - Deterministic은 그 함수가 일관성 있는 결과를 리턴함을 선언 하는 것일 뿐 일관성이 보장되는 것이 아님.

**복잡한 함수 로직을 풀어 SQL로 구현**